# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [2]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BenchmarkTools

Threads.nthreads() == 1 && 
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`");
println("Number of threads: $(Threads.nthreads())")

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


Number of threads: 4


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [3]:
function LoopFields(c, model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(c, model, Δmax) = Spectrum(LoopFields(c, model), Δmax, interchiral=true);

function solve(;indices, signature, parity, even_spin=(), benchmark=false)
    ext_fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(ext_fields..., Δmax=Δmax)
    SOn = LoopSpectrum(c, :On, Δmax)
    Seven = Spectrum([V for V in SOn.fields if spin(V) % 2 == 0], Δmax, interchiral=true)

    specs = (; (chan =>
        if chan in even_spin
            Seven
        else
            SOn
        end
                for chan in (:s, :t, :u)
    )...)

    chan_spectra = ChannelSpectra(
        co, specs, signature;
        interchiral=true, Δmax=Δmax, parity=1
    )

    sys = BootstrapSystem(chan_spectra)
    evaluate_blocks!(sys)
    compute_linear_system!(sys)
    solve!(sys)
    if benchmark
        println("precomputation of blocks coefficients")
        @btime ChannelSpectra(
            $co, $SOn, $signature;
            interchiral=true, Δmax=Δmax, parity=1,
            exclude=(u=[Field(c, r=2, s=0)],)
        )
        println("time to compute all positions' cache")
        @btime BootstrapSystem($chan_spectra)
        println("time to evaluate blocks")
        @btime evaluate_blocks!($sys)
        println("time for the linear solver")
        @btime solve!($sys)
    end
    return sys
end

solve (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [4]:
Δmax, prec = 30, 25
setprecision(BigFloat, prec, base=10);
c = CC(β=1 / (big"0.8" + big"0.1" * im))
ind = ((1 // 2, 0), (1 // 2, 0), (1 // 2, 0), (1 // 2, 0))
sig = (s=0, t=1, u=1)
sol = solve(indices=ind, signature=sig, parity=1, even_spin=(:s,), benchmark=false)
# println("total time to setup and solve the crossing equations:")
# @btime solve(indices=ind, signature=sig, parity=1, even_spin=(:s,))
println(sol.structure_constants)

70×4 DataFrame
 Row │ Channel  Field          StructureConstant           RelativeError 
     │ Symbol   String         Complex                     Float32       
─────┼───────────────────────────────────────────────────────────────────
   1 │ s        V_{<1, 1>}              1.0+0.0im            0.0
   2 │ s        V_{(1, 0)}        -0.182995+0.243693im       5.42432e-20
   3 │ s        V_{(2, 0)}     -0.000165018+0.000981007im    1.99491e-17
   4 │ s        V_{(2, 1)}      0.000133433-0.000458461im    6.55796e-18
   5 │ s        V_{(3, 0)}      2.60868e-08+1.31146e-08im    1.41853e-12
   6 │ s        V_{(3, 2//3)}  -1.26635e-08-6.18514e-09im    1.20763e-12
   7 │ s        V_{(4, 0)}     -5.24347e-14+1.08689e-15im    4.47935e-6
   8 │ s        V_{(4, 1//2)}   3.75941e-14+1.58324e-14im    3.98313e-6
   9 │ s        V_{(4, 1)}      -2.0752e-14-1.4879e-14im     2.97513e-6
  10 │ s        V_{(5, 0)}      2.79135e-18+1.54486e-18im    1.12884
  11 │ s        V_{(5, 2//5)}   2.41542e-18+9.71

In [5]:
@btime BootstrapVirasoro.compute_reference!(sol)

  3.940 s (24838677 allocations: 1.07 GiB)


In [6]:
total_dimension(Field(c, r=6, s=0))
sol.spectra.s.Δmax

30.0 + 0.0im

Benchmarks:
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [5]:
ind = ((1, 0), (1, 0), (1//2, 0), (1//2, 0))
sig = (s=0, t=3//2, u=3//2)
println(solve(indices=ind, signature=sig, parity=1, even_spin=(:s,)).structure_constants)

56×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        V_{<1, 1>}                    1.0+0.0im            0.0
   2 │ s        V_{(1, 0)}              -0.256792+0.389886im       8.5896e-19
   3 │ s        V_{(2, 0)}             0.00281023-0.00250618im     1.79582e-16
   4 │ s        V_{(2, 1)}           -7.02072e-21-1.42797e-20im    1.03753
   5 │ s        V_{(3, 0)}           -1.19812e-06-9.08135e-07im    2.0874e-12
   6 │ s        V_{(3, 2//3)}         8.21729e-07+4.50803e-07im    2.85204e-13
   7 │ s        V_{(4, 0)}           -1.17949e-12+4.35214e-13im    1.81557e-5
   8 │ s        V_{(4, 1//2)}         6.59511e-13-2.50729e-13im    4.22059e-6
   9 │ s        V_{(4, 1)}            1.29889e-18-4.01213e-19im    3.59638
  10 │ s        V_{(5, 0)}            1.99037e-16+4.39

In [6]:
sig = (s=1, t=1//2, u=3//2)
println(solve(indices=ind, signature=sig, parity=1).structure_constants)

68×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        V_{(1, 0)}                    1.0+0.0im            0.0
   2 │ s        V_{(1, 1)}               0.502183-0.0830317im      3.63093e-20
   3 │ s        V_{(2, 0)}             0.00239155+0.0255139im      1.90449e-18
   4 │ s        V_{(2, 1//2)}          0.00273704+0.0200265im      1.56372e-18
   5 │ s        V_{(2, 1)}            3.54491e-21+3.70845e-20im    0.932588
   6 │ s        V_{(3, 0)}           -6.40312e-07-1.92164e-06im    6.58496e-13
   7 │ s        V_{(3, 1//3)}        -7.07647e-07-9.43683e-07im    7.51062e-13
   8 │ s        V_{(3, 2//3)}        -6.00257e-07+2.35486e-07im    1.30508e-12
   9 │ s        V_{(3, 1)}           -7.64856e-19-5.78058e-19im    0.947315
  10 │ s        V_{(4, 0)}            1.40202e-1

In [7]:
sig = (s=1, t=3//2, u=1//2)
println(solve(indices=ind, signature=sig, parity=1).structure_constants)

68×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        V_{(1, 0)}                    1.0+0.0im            0.0
   2 │ s        V_{(1, 1)}              -0.502183+0.0830317im      7.69278e-19
   3 │ s        V_{(2, 0)}             0.00239155+0.0255139im      4.54232e-17
   4 │ s        V_{(2, 1//2)}         -0.00273704-0.0200265im      6.99145e-18
   5 │ s        V_{(2, 1)}            3.57573e-20+1.44965e-20im    7.39925
   6 │ s        V_{(3, 0)}           -6.40312e-07-1.92164e-06im    1.52966e-12
   7 │ s        V_{(3, 1//3)}         7.07647e-07+9.43683e-07im    3.33267e-12
   8 │ s        V_{(3, 2//3)}        -6.00257e-07+2.35486e-07im    3.64602e-12
   9 │ s        V_{(3, 1)}           -4.04364e-19-2.88457e-19im    1.24155
  10 │ s        V_{(4, 0)}            1.40055e-12+

# Torus one point functions

In [8]:
V1 = Field(c, r=1, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1//2, t=0, u=1//2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
println(sys.structure_constants)

70×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        V_{(1//2, 0)}                 1.0+0.0im               0.0
   2 │ s        V_{(3//2, 0)}            0.260415-1.76491im           1.25568
   3 │ s        V_{(3//2, 2//3)}         -4.09692+2.84072im           0.438296
   4 │ s        V_{(5//2, 0)}            -19231.2-8970.15im           4.16523
   5 │ s        V_{(5//2, 2//5)}          15525.1+7166.17im           4.15426
   6 │ s        V_{(5//2, 4//5)}         -5878.44-2732.5im            4.15986
   7 │ s        V_{(7//2, 0)}           1.849e+07+3.19369e+07im       1.61209
   8 │ s        V_{(7//2, 2//7)}     -1.66551e+07-2.87377e+07im       1.61207
   9 │ s        V_{(7//2, 4//7)}       1.1522e+07+1.98157e+07im       1.61197
  10 │ s        V_{(7//2, 6//7)}     -4.10631e

In [ ]:
Δmax, prec = 30, 25
setprecision(BigFloat, prec, base=10);
c = CC(β=1 / (big"0.8" + big"0.1" * im))
DG = DoubleGamma(c.β)
@btime DG(big"0.4"+big"0.1"*im)

  18.527 ms (106050 allocations: 4.72 MiB)


2.09190949932900071267021706855002182180092 - 0.31712440179626803895784051274373588364991im